In [1]:
import os
import sys
import django

print(sys.path)
# https://docs.djangoproject.com/en/5.0/topics/async/#async-safety
# ORMでの操作は非同期では安全ではないらしく、JupyterNotebook上など非同期なコンテキストから実行される場合、実行が制限されてしまう。
# 非同期なコンテキストからORMを使うためには、DJANGO_ALLOW_ASYNC_UNSAFE環境変数を設定する必要がある
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "app.settings")
django.setup()


['/django-work/notebook', '/django-work/app', '/usr/local/lib/python312.zip', '/usr/local/lib/python3.12', '/usr/local/lib/python3.12/lib-dynload', '', '/usr/local/lib/python3.12/site-packages']


In [2]:
from polls.models import Choice, Question
from django.utils import timezone

In [3]:
Question.objects.all()

<QuerySet []>

In [4]:
# データの追加
q = Question(question_text="What's new?", pub_date=timezone.now())
q.save()
print(q.id)
print(q.question_text)
print(q.pub_date)

1
What's new?
2024-03-06 07:32:07.991378+00:00


In [5]:
# データの変更
q.question_text = "What's up?"
q.save()
print(q.id)
print(q.question_text)
print(q.pub_date)

1
What's up?
2024-03-06 07:32:07.991378+00:00


In [6]:
Question.objects.all()

<QuerySet [<Question: What's up?>]>

In [7]:
# idを指定してデータを取得
Question.objects.filter(id=1)

<QuerySet [<Question: What's up?>]>

In [8]:
# テキストの前方一致でフィルタリング
Question.objects.filter(question_text__startswith="What")

<QuerySet [<Question: What's up?>]>

In [9]:
from django.utils import timezone

# 今年に投稿された質問を取得
current_year = timezone.now().year
Question.objects.filter(pub_date__year=current_year).first()

<Question: What's up?>

In [10]:
# 存在しないID
Question.objects.get(id=10000)

DoesNotExist: Question matching query does not exist.

In [11]:
# モデルに実装したメソッド(was_published_recently)を呼び出す
q = Question.objects.get(pk=1)
q.was_published_recently()

True

In [12]:
# Questionテーブルの子テーブルであるChoiceテーブルのデータを取得
q = Question.objects.get(pk=1)
q.choice_set.all()

<QuerySet []>

In [13]:
# Choisceテーブルにデータを追加
q.choice_set.create(choice_text="Not much", votes=0)
q.choice_set.create(choice_text="The sky", votes=0)
q.choice_set.create(choice_text="Just hacking again", votes=0)

<Choice: Just hacking again>

In [14]:
# Questionテーブルの子テーブルであるChoiceテーブルのデータを取得
q = Question.objects.get(pk=1)
print(q.choice_set.all())
print(q.choice_set.count())


<QuerySet [<Choice: Not much>, <Choice: The sky>, <Choice: Just hacking again>]>
3


In [15]:
# 投稿日が今年のQuestionに紐づくChoiseを取得
Choice.objects.filter(question__pub_date__year=current_year)

<QuerySet [<Choice: Not much>, <Choice: The sky>, <Choice: Just hacking again>]>

In [16]:
# Choiceテーブルのデータを削除
c = q.choice_set.filter(choice_text__startswith="Just hacking")
c.delete()

(1, {'polls.Choice': 1})